In [1]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from datetime import datetime, date, time
from datetime import timedelta
import matplotlib as mpl
import matplotlib.pyplot as plt
import holidays 

# set float display default
pd.set_option('display.float_format', lambda x: '%.2f' % x)


# Read the Data

In [2]:
DataDictionary = pd.read_excel('DataDictionary.xlsx')
TCU_HolidayList = pd.read_csv('TCU_HolidayList.csv')
training = pd.read_csv('Training.csv')
supplemental = pd.read_csv('Training_SupplementalFields.csv')

## Change time variable

In [3]:
training['BusinessDate'] = pd.to_datetime(training['BusinessDate'])
supplemental['BusinessDate'] = pd.to_datetime(supplemental['BusinessDate'])
TCU_HolidayList['HolidayDate'] = pd.to_datetime(TCU_HolidayList['HolidayDate'])

In [4]:
training.head()

,BusinessDate,BranchID,CashOrdersReceived,CashBalance_StartOfDay,TotalCashUsed
0,2020-07-31,99,-,"775,054.00","-43,718.13"
1,2020-07-30,99,-,"1,036,464.13","261,410.14"
2,2020-07-29,99,"400,540.11","1,141,386.84","104,922.70"
3,2020-07-28,99,-,"864,871.20","124,024.48"
4,2020-07-27,99,-,"923,948.57","59,077.37"


In [5]:
training.shape

(171230, 5)

In [6]:
supplemental.head()

,BusinessDate,BranchID,CashToVault,CashUnder,CashOver,TotalCashDisbursed,TotalCashOpeningBalance,TotalCashReceivedAllSources
0,2020-07-31,99,-,-,-,"1,196,392.62","1,095,684.09","875,762.53"
1,2020-07-30,99,-,-,-,-,"1,036,464.13",-
2,2020-07-29,99,-,-,-,-,"1,141,386.84",-
3,2020-07-28,99,-,-,-,-,"864,871.20",-
4,2020-07-27,99,-,-,-,-,"923,948.57",-


In [7]:
supplemental.shape

(171230, 8)

In [8]:
TCU_HolidayList.head()

,HolidayDate,HolidayName
0,2020-12-25,Christmas Day
1,2020-12-24,Christmas Eve (TCU only)
2,2020-11-27,Day after T-Day (Fri - TCU only)
3,2020-11-26,Thanksgiving
4,2020-11-11,Veterans Day


## Create a full holiday list

In [9]:
year = range(2010,2022)
us_holidays = holidays.UnitedStates(years=year)
us_holidays= pd.DataFrame(list(us_holidays.items()),columns = ['HolidayDate','Holiday']) 
us_holidays['HolidayDate']=pd.to_datetime(us_holidays['HolidayDate'])
us_holidays.sort_values(['HolidayDate'], inplace=True)

In [10]:
full_holidays = pd.merge(TCU_HolidayList, us_holidays, left_on='HolidayDate', right_on='HolidayDate', how='outer')
full_holidays['Holiday'].fillna(full_holidays['HolidayName'], inplace= True)
full_holidays=full_holidays[['HolidayDate','Holiday']]

In [11]:
full_holidays.head()

,HolidayDate,Holiday
0,2020-12-25,Christmas Day
1,2020-12-24,Christmas Eve (TCU only)
2,2020-11-27,Day after T-Day (Fri - TCU only)
3,2020-11-26,Thanksgiving
4,2020-11-11,Veterans Day


In [12]:
full_holidays.shape , TCU_HolidayList.shape

((150, 2), (123, 2))

In [13]:
# rename holidays
def holiday_rename(series):
    if series == "Christmas Eve (TCU only)":
        return "Christmas Eve"
    if series == "Day after T-Day (Fri - TCU only)":
        return "Day After T-Day (TCU only)"
    else: 
        return series
full_holidays['Holiday'] = full_holidays['Holiday'].apply(holiday_rename) 

In [14]:
#save the full_holidays dataset for model use
full_holidays.to_csv('full_holidays.csv', index=False)

## Add holidays before and after to account for additional closing dates

In [15]:
before_holidays = pd.DataFrame()
after_holidays = pd.DataFrame()

before_holidays['before'] = full_holidays['HolidayDate']-timedelta(days=1)
before_holidays['before_holiday'] = full_holidays['Holiday'] + '_Before'
after_holidays['after'] = full_holidays['HolidayDate'] + timedelta(days=1)
after_holidays['after_holiday'] = full_holidays['Holiday'] + '_After'

In [16]:
before_holidays.head()

,before,before_holiday
0,2020-12-24,Christmas Day_Before
1,2020-12-23,Christmas Eve_Before
2,2020-11-26,Day After T-Day (TCU only)_Before
3,2020-11-25,Thanksgiving_Before
4,2020-11-10,Veterans Day_Before


In [17]:
after_holidays.head()

,after,after_holiday
0,2020-12-26,Christmas Day_After
1,2020-12-25,Christmas Eve_After
2,2020-11-28,Day After T-Day (TCU only)_After
3,2020-11-27,Thanksgiving_After
4,2020-11-12,Veterans Day_After


# Merge and clean data

In [18]:
df = pd.merge(training, supplemental, how='left',
        left_on=['BusinessDate', 'BranchID'], right_on=['BusinessDate','BranchID'])

## remove comma, - and () from df 

In [19]:
df.replace(',','',regex=True, inplace=True)
df.replace(' -   ','',regex=True, inplace=True)
df.replace('[(]', '-',regex=True, inplace=True)
df.replace('[)]', '',regex=True, inplace=True)

In [20]:
# merge with holiday list
df = pd.merge(df, full_holidays, how='left',
        left_on='BusinessDate', right_on='HolidayDate')

# merge with before holiday list
df = pd.merge(df, before_holidays, how='left',
        left_on='BusinessDate', right_on='before')

# merge with after holiday list
df = pd.merge(df, after_holidays, how='left',
        left_on='BusinessDate', right_on='after')

#remove white spaces from column names
df.columns = df.columns.str.replace(' ', '')


## Generate year, month, week

In [21]:
df['Year']=df['BusinessDate'].dt.year
df['Month']=df['BusinessDate'].dt.month
df['Day_name']= df['BusinessDate'].dt.day_name()

## Sort data

In [22]:
# sort branch and business date
df.sort_values(by=['BranchID','BusinessDate'], inplace=True)
# set businessdate to index
df.set_index('BusinessDate', inplace=True)

In [23]:
df.head()

,BranchID,CashOrdersReceived,CashBalance_StartOfDay,TotalCashUsed,CashToVault,CashUnder,CashOver,TotalCashDisbursed,TotalCashOpeningBalance,TotalCashReceivedAllSources,HolidayDate,Holiday,before,before_holiday,after,after_holiday,Year,Month,Day_name
BusinessDate,,,,,,,,,,,,,,,,,,,
2010-01-01,99,,1195235.33,48082.66,,,,,1195235.33,,2010-01-01,New Year's Day,NaT,NaN,NaT,NaN,2010,1,Friday
2010-01-02,99,,1147152.68,,,,,,1147152.68,,NaT,NaN,NaT,NaN,2010-01-02,New Year's Day_After,2010,1,Saturday
2010-01-03,99,,1147152.68,92262.26,,,,,1147152.68,,NaT,NaN,NaT,NaN,NaT,NaN,2010,1,Sunday
2010-01-04,99,,1054890.42,64107.06,,,,,1054890.42,,NaT,NaN,NaT,NaN,NaT,NaN,2010,1,Monday
2010-01-05,99,,990783.35,12372.77,,,,,990783.35,,NaT,NaN,NaT,NaN,NaT,NaN,2010,1,Tuesday


In [24]:
df.shape

(171230, 19)

In [25]:
df.to_csv('combined_data.csv', index=True)